<a href="https://colab.research.google.com/github/miguel10028/IA_UdeA/blob/main/03%20-%20modelo%20con%20preprocesado%20mediante%20Pipeline%2C%20ColumnTransformer%20y%20SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrega Final
En este entrega tendrás que someter tu solución a Kaggle. CADA MIEMBRO DE CADA GRUPO TIENE QUE SOMETER SU PROPIA SOLUCIÓN, aunque sea la misma que la del resto de componentes del grupo.

Además del contenido de las entregas anteriores tendrás que añadir a tu repositorio github

- README.md con los nombres, cédulas y progama en el que está inscrito cada participante y con los siguientes enlaces y con el enlace al video de la - entrega 2 y el enlace al video de esta entrega.

- **Video subido a YouTube y enlazado en README.md**

    - Elabora un vídeo de entre 3mins y 4mins.

    - En el video ha de explicar brevemente la aproximación a la solución enviada a Kaggle y ha de mostrar vuestra posición en el ranking compartiendo la pantalla mientras se consulta el leaderboard de Kaggle. Esta posición, por supuesto, puede que no sea la definitiva si hay equipos que envian sus soluciones después.

- 99 - modelo solución.ipynb - Un notebook con tu solución completa, desde cargar train.csv hasta generar el archivo que se envía a Kaggle. Mira las NOTES 05.03 para tener un ejemplo.

- al menos dos notebooks adicionales con aproximaciones distantas a la de la solución, porque usan distintos modelos, o estrategias de limpieza o preprocesado, etc.

Los criterios de evaluación son los siguientes:

- CLARIDAD: los notebooks contienen textos que explican brevemente lo que se está haciendo
- COMPLECIÓN: la entrega contiene todos los notebooks que se describieron justo arriba.
- REPRODUCIBILIDAD: la solución que genera el notebook 99 - modelo solución.ipynb coincide con la enviada a Kaggle.

Al final tu repositorio habrá de tener la siguiente estructura:  

    +-  README-md
    +-  01 - exploración.ipynb
    +-  02 - preprocesado.ipynb
    +-  03 - modelo con preprocesado de tal forma y SVM.ipynb
    +-  04 - modelo con preprocesado de otra forma y Random Forest.ipynb
    +-  99 - modelo solución.ipynb
    
El nombrado de los archivos ES ESTRICTO. Tiene que haber un fichero README.md y los notebooks 01, 02 y 99 nombrados EXACTAMENTE IGUAL que se indica arriba. Los notebooks 03, 04 y posteriores (si tienes más), han de ser nombrados con ese esquema (número consecutivo - nombre_que_escojes.ipynb). Cualquier archivo con otro nombre será ignorado.

Cada miembro del grupo ES RESPONSABLE DE SU PROPIA ENTREGA, independientemente de si sus compañeros de grupo entregan o no. Esto incluye rellenar el formulario y tener un github personal, aunque el contenido sea el mismo que el del resto de los compañeros de grupo.

Si cambias de grupo o te añades a un grupo que ya exista tienes que enviar un correo a raul.ramos@udea.edu.co, felipe.rodrigueza@udea.edu.co informándolo con copia a tus nuevos compañeros indicando las razones del cambio y cuál será tu rol en el nuevo grupo. Tus nuevos compañeros deberán de responder ese correo autorizandote a que formes parte del grupo.

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


## Download data directly from Kaggle

- create a file `kaggle.json` with your authentication token (in kaggle $\to$ click user icon on top-right $\to$ settings $\to$ API create new token)
- upload it to this notebook workspace
- run the following cell

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 70% 14.0M/20.1M [00:00<00:00, 146MB/s]
100% 20.1M/20.1M [00:00<00:00, 172MB/s]


## unzip and inspect data

In [ ]:
!unzip udea*.zip > /dev/null


In [ ]:
# Importamos librerías
import os
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, mean_absolute_error, accuracy_score, mean_squared_error, precision_score, f1_score, recall_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Lectura del archivo "train.csv"

nombre_archivo = "train.csv"

#Creamos el DataFrame
df = pd.read_csv(nombre_archivo)

# Mostrar un breve resumen del archivo "train.csv"
df.head(10)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,Más de 7 millones,Menos de 10 horas,Estrato 5,Si,Educación profesional completa,Secundaria (Bachillerato) completa,No,medio-alto
6,47159,20183,INGENIERIA MECANICA,HUILA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Educación profesional incompleta,Técnica o tecnológica completa,Si,alto
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Entre 11 y 20 horas,Estrato 2,Si,Primaria incompleta,Secundaria (Bachillerato) incompleta,Si,medio-bajo
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 1,Si,Secundaria (Bachillerato) completa,Educación profesional incompleta,Si,medio-bajo
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 5,Si,Postgrado,Postgrado,Si,alto


In [ ]:
#Información dimensiones archivo "train.csv":
print("\nTamaño del archivo:",
      "\nFilas:", df.shape[0],
      "\nColumnas:", df.shape[1])

#Verificar datos faltantes para la limpieza y el preprocesado
print("\nDatos Faltantes por columna:\n",df.isnull().sum())


Tamaño del archivo: 
Filas: 692500 
Columnas: 12

Datos Faltantes por columna:
 ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_EDUCACIONMADRE               23664
ESTU_PAGOMATRICULAPROPIO           6498
RENDIMIENTO_GLOBAL                    0
dtype: int64


# Detalles del codigo propuesto para SVM

Detalles del código:
- **preprocess_data(df):** Esta función prepara los datos para el modelo, transformando las variables categóricas con un codificador OneHotEncoder y las ordinales con un OrdinalEncoder. También aplica un SimpleImputer para manejar valores faltantes.

- **Optimización con GridSearchCV:** Se ha implementado GridSearchCV para optimizar los hiperparámetros del modelo SVM, incluyendo el parámetro C, el kernel (lineal y RBF) y el gamma. Esto mejora la precisión sin afectar demasiado el tiempo de entrenamiento al explorar diferentes configuraciones de parámetros.

- **PCA:** La reducción de dimensionalidad con PCA se realiza para reducir el número de características y acelerar el modelo. Puedes ajustar el valor de n_components según sea necesario.

- **Escalado de datos:** Se aplica un escalado de las características antes de entrenar el modelo, lo cual es esencial cuando se usan métodos como SVM.

# SVM (Support Vector Machine)

In [ ]:
# Preprocesamiento previo al modelo
def preprocess_data(df):
    X = df.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
    y = df['RENDIMIENTO_GLOBAL']

    # Características categóricas y ordinales
    categorical_features = [
        'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
        'FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO'
    ]
    ordinal_features = [
        'ESTU_VALORMATRICULAUNIVERSIDAD', 'FAMI_ESTRATOVIVIENDA',
        'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE',
        'ESTU_HORASSEMANATRABAJA'
    ]

    # Preprocesamiento
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    ordinal_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal', OrdinalEncoder()),
        ('scaler', StandardScaler())
    ])
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features),
            ('ord', ordinal_transformer, ordinal_features)
        ])

    X_preprocessed = preprocessor.fit_transform(X)
    return X_preprocessed, y


In [ ]:
# Modelo SVM Optimizado
def svm_classifier_optimized(X, y, param_C, n_components=20):
    # Reducción de dimensionalidad con PCA
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)

    # Convierte y a etiquetas categóricas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    skf = StratifiedKFold(n_splits=4)
    resultados = pd.DataFrame()
    conf_matrix_list_of_arrays = np.zeros((len(np.unique(y_encoded)), len(np.unique(y_encoded))))

    idx = 0
    for C_value in param_C:
        EficienciaTrain = []
        EficienciaVal = []
        Macc = []
        Mpre = []
        Mrec = []
        Mf1 = []

        print(f"Training with SVM (C={C_value})...")

        for train_index, test_index in skf.split(X_reduced, y_encoded):
            X_train, X_test = X_reduced[train_index], X_reduced[test_index]
            y_train, y_test = y_encoded[train_index], y_encoded[test_index]

            # Modelo SVM Lineal más rápido
            model = LinearSVC(C=C_value, random_state=0, max_iter=10000, dual=False)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            y_pred_train = model.predict(X_train)

            conf_matrix = confusion_matrix(y_test, y_pred)
            conf_matrix_list_of_arrays += conf_matrix

            EficienciaTrain.append(np.mean(y_pred_train.ravel() == y_train.ravel()))
            EficienciaVal.append(np.mean(y_pred.ravel() == y_test.ravel()))
            Macc.append(accuracy_score(y_test, y_pred))
            Mpre.append(precision_score(y_test, y_pred, average="weighted"))
            Mrec.append(recall_score(y_test, y_pred, average="weighted"))
            Mf1.append(f1_score(y_test, y_pred, average="weighted"))

        # Resultados por parámetro C
        resultados.loc[idx, 'C (parámetro de regularización)'] = C_value
        resultados.loc[idx, 'eficiencia de entrenamiento'] = np.mean(EficienciaTrain)
        resultados.loc[idx, 'desviación estándar entrenamiento'] = np.std(EficienciaTrain)
        resultados.loc[idx, 'eficiencia de prueba'] = np.mean(EficienciaVal)
        resultados.loc[idx, 'Intervalo de confianza (prueba)'] = np.std(EficienciaVal)
        resultados.loc[idx, 'accuracy real'] = np.mean(Macc)
        resultados.loc[idx, 'precision_score'] = np.mean(Mpre)
        resultados.loc[idx, 'recall_score'] = np.mean(Mrec)
        resultados.loc[idx, 'f1_score'] = np.mean(Mf1)
        idx += 1

    return resultados, conf_matrix_list_of_arrays

In [ ]:
# Preprocesamiento
X, y = preprocess_data(df)

# Parámetros para SVM
param_C = [0.1, 1, 10]

# Entrenamiento con SVM
resultados, matriz_confusion = svm_classifier_optimized(X, y, param_C)

# Resultados
print(resultados)
print("Matriz de Confusión Total:\n", matriz_confusion)


Training with SVM (C=0.1)...
Training with SVM (C=1)...
Training with SVM (C=10)...
   C (parámetro de regularización)  eficiencia de entrenamiento  \
0                              0.1                     0.372220   
1                              1.0                     0.372224   
2                             10.0                     0.372225   

   desviación estándar entrenamiento  eficiencia de prueba  \
0                           0.000295              0.372211   
1                           0.000292              0.372212   
2                           0.000292              0.372211   

   Intervalo de confianza (prueba)  accuracy real  precision_score  \
0                         0.000588       0.372211         0.344329   
1                         0.000593       0.372212         0.344330   
2                         0.000593       0.372211         0.344327   

   recall_score  f1_score  
0      0.372211  0.321707  
1      0.372212  0.321709  
2      0.372211  0.321708  
Matri

# Propuesta 2 (SVM)
Se modificaron los parametros param_C = [0.1, 1, 10] y se implemento prueba con kernel = 'linear'

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC  # Cambiamos a SVC para usar diferentes kernels
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd

# Modelo SVM Optimizado
def svm_classifier_optimized(X, y, param_C, kernel='linear', tol=1e-4, max_iter=5000, n_components=10):
    # Reducción de dimensionalidad con PCA
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)

    # Convierte y a etiquetas categóricas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    skf = StratifiedKFold(n_splits=3)
    resultados = pd.DataFrame()

    idx = 0
    for C_value in param_C:
        EficienciaTrain, EficienciaVal, Macc, Mpre, Mrec, Mf1 = [], [], [], [], [], []
        print(f"Training with SVM (C={C_value}, kernel={kernel})...")

        for train_index, test_index in skf.split(X_reduced, y_encoded):
            X_train, X_test = X_reduced[train_index], X_reduced[test_index]
            y_train, y_test = y_encoded[train_index], y_encoded[test_index]

            # Modelo SVM con kernel ajustable
            model = SVC(C=C_value, kernel=kernel, tol=tol, max_iter=max_iter, random_state=0)
            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            y_pred_train = model.predict(X_train)

            EficienciaTrain.append(np.mean(y_pred_train == y_train))
            EficienciaVal.append(np.mean(y_pred == y_test))
            Macc.append(accuracy_score(y_test, y_pred))
            Mpre.append(precision_score(y_test, y_pred, average="weighted"))
            Mrec.append(recall_score(y_test, y_pred, average="weighted"))
            Mf1.append(f1_score(y_test, y_pred, average="weighted"))

        # Guardar resultados promedio por cada valor de C
        resultados.loc[idx, 'C (parámetro de regularización)'] = C_value
        resultados.loc[idx, 'eficiencia de entrenamiento'] = np.mean(EficienciaTrain)
        resultados.loc[idx, 'desviación estándar entrenamiento'] = np.std(EficienciaTrain)
        resultados.loc[idx, 'eficiencia de prueba'] = np.mean(EficienciaVal)
        resultados.loc[idx, 'Intervalo de confianza (prueba)'] = np.std(EficienciaVal)
        resultados.loc[idx, 'accuracy real'] = np.mean(Macc)
        resultados.loc[idx, 'precision_score'] = np.mean(Mpre)
        resultados.loc[idx, 'recall_score'] = np.mean(Mrec)
        resultados.loc[idx, 'f1_score'] = np.mean(Mf1)
        idx += 1

    return resultados

In [ ]:
# Preprocesamiento
X, y = preprocess_data(df)

# Parámetros para SVM
param_C = [0.01, 0.1, 1]
kernel = 'linear'  # Puedes probar 'linear', 'rbf', 'poly'

# Entrenamiento con SVM
resultados = svm_classifier_optimized(X, y, param_C, kernel=kernel, tol=1e-3, max_iter=2000)

# Resultados
print(resultados)

Training with SVM (C=0.01, kernel=linear)...
Training with SVM (C=0.1, kernel=linear)...
Training with SVM (C=1, kernel=linear)...
   C (parámetro de regularización)  eficiencia de entrenamiento  \
0                             0.01                     0.317196   
1                             0.10                     0.318962   
2                             1.00                     0.267866   

   desviación estándar entrenamiento  eficiencia de prueba  \
0                           0.002641              0.316936   
1                           0.016423              0.319092   
2                           0.021615              0.267964   

   Intervalo de confianza (prueba)  accuracy real  precision_score  \
0                         0.003627       0.316936         0.332874   
1                         0.015567       0.319092         0.334521   
2                         0.022085       0.267964         0.296986   

   recall_score  f1_score  
0      0.316936  0.291929  
1      0.31909